In [1]:
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, ReLU, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.keras.__version__)

2.9.0


In [2]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            #Appending activity class to each row in the dataset
            dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            return grouped_rows
        except Exception as e:
            print(e)

In [ ]:
#For short term prediction, we need a sample size of 20(10 frames input sequance, 10 frames predicted sequance)
sampled_data = Dataset_Preprocessing('./H3.6csv', sample_size=30).read_dataset()

In [ ]:
total_batches = sampled_data.shape[0]
sampled_data = sampled_data[:total_batches-(total_batches%100)]

In [ ]:
def split_to_features_labels(dataset, input_sequance_size=10) :
    assert input_sequance_size < dataset.shape[1], f"input sequance should be smaller than the total sample size"
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :64]
    
    return features, labels

In [ ]:
sampled_dataX, sampled_dataY = split_to_features_labels(sampled_data, input_sequance_size=10)

In [ ]:
print('Total Samples: {}'.format(sampled_dataY.shape[0]))
print('Total Frames: {}'.format(sampled_dataY.shape[1]))
print('Total Features: {}'.format(sampled_dataY.shape[2]))

In [39]:
class GloGenAutoRegressive(Model_):
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200,
                 output_diminsion=64, activation='relu', input_sequance_length=10,
                 output_sequance_length=10, batch_size=100):
      super(GloGenAutoRegressive, self).__init__()
      self.batch_size = batch_size
      self.output_diminsion = output_diminsion
      self.encoder = LSTM(enocder_hidden_state, return_state=True, return_sequences=True)
      self.decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
      self.dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 
      assert output_sequance_length % input_sequance_length == 0,f"{output_sequance_length} is not divisible by {input_sequance_length}"
      self.output_sequance_length = output_sequance_length
      self.input_sequance_length = input_sequance_length  

    def call(self, inputs):
      next_inputs = inputs
      class_priors = inputs[:,:,-10:]
      num_time_slices = int(self.output_sequance_length/self.input_sequance_length)
      full_predicted_sequance = tf.zeros([self.batch_size,0,64])
      #full_predicted_sequance = tf.reshape((), (0, num_time_slices))
      print(full_predicted_sequance.shape)
      #for loop for predicting the t*n sequance autoregressively from previously predicted sequance
      for time_slice in range(num_time_slices) :
        print(num_time_slices)
        encoder_outputs, state_h, state_c = self.encoder(next_inputs)
        encoder_states = [state_h, state_c]
        output, _, _ = self.decoder(encoder_outputs, initial_state=encoder_states)
        print(next_inputs.shape)
        next_inputs = self.dense_layer(output)
        print(next_inputs.shape)
        #Appending the class priors to the next inputs
        full_predicted_sequance = tf.concat((full_predicted_sequance, next_inputs), axis=1)
        print(full_predicted_sequance.shape)
        next_inputs = tf.concat((next_inputs, class_priors), axis=2)
        print(next_inputs.shape)
      return full_predicted_sequance

In [15]:
class GloGen(Model_):
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200, output_diminsion=64, activation='relu'):
        super(GloGen, self).__init__()
        self.encoder = LSTM(enocder_hidden_state, return_state=True, return_sequences=True)
        self.decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
        self.dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 

    def call(self, inputs):
        encoder_outputs, state_h, state_c = self.encoder(inputs)
        encoder_states = [state_h, state_c]
        output, _, _ = self.decoder(encoder_outputs, initial_state=encoder_states)
        output = self.dense_layer(output)
        return output

In [27]:
class JointLoss() :
  def __init__(self, lambda1=0.5, lambda2=0.5) :
    self.lambda1 = lambda1
    self.lambda2 = lambda2

  def loss_joint(self, predicted_sequance_batch, target_sequance_batch) :
      diff_norm_2 = tf.math.reduce_sum(tf.square(tf.subtract(predicted_sequance_batch, target_sequance_batch)), axis=2)
      return tf.reduce_sum(diff_norm_2, axis=1) 

  def loss_motion_flow(self, predicted_sequance_batch, target_sequance_batch) :
      predictions_tomporal_diffs = tf.experimental.numpy.diff(predicted_sequance_batch, axis=1)
      real_tomporal_diffs = tf.experimental.numpy.diff(target_sequance_batch, axis=1)
      prediction_motion_flow_diff_norm_2 = tf.reduce_sum(tf.square(tf.subtract(predictions_tomporal_diffs, real_tomporal_diffs)), axis=2)
      return tf.reduce_sum(prediction_motion_flow_diff_norm_2, axis=1)


  def total_loss(self, target_sequance_batch, predicted_sequance_batch) :
      joints_loss = self.loss_joint(predicted_sequance_batch, target_sequance_batch)
      motion_flow_loss = self.loss_motion_flow(predicted_sequance_batch, target_sequance_batch)
      return self.lambda1*joints_loss + self.lambda2*motion_flow_loss

In [59]:
def run_experiment(learning_rate=0.002, lambda1=0.5, lambda2=0.5, use_mse=False
                   , metrics=None, batch_size=100, epochs=50, validation_split=0.2) :
  glogen_model = GloGenAutoRegressive(batch_size=batch_size, output_sequance_length=20)
  if use_mse :
    loss_function = tf.keras.losses.mean_squared_error
  else :
    loss_function = JointLoss(lambda1=lambda1, lambda2=lambda2).total_loss
  glogen_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                       loss=loss_function, metrics=metrics)
  history = glogen_model.fit(sampled_dataX, sampled_dataY,
          batch_size=batch_size,
          epochs=epochs)
  return history

In [60]:
history = run_experiment(epochs=10, lambda1=0.8, lambda2=0.2, metrics=[tf.keras.losses.mean_absolute_percentage_error])

Epoch 1/10
(100, 0, 64)
2
(100, 10, 74)
(100, 10, 64)
(100, 10, 64)
(100, 10, 74)
2
(100, 10, 74)
(100, 10, 64)
(100, 20, 64)
(100, 10, 74)
(100, 0, 64)
2
(100, 10, 74)
(100, 10, 64)
(100, 10, 64)
(100, 10, 74)
2
(100, 10, 74)
(100, 10, 64)
(100, 20, 64)
(100, 10, 74)
513/513 [==============================] - 126s 233ms/step - loss: 167247600.0000 - mean_absolute_percentage_error: 78.4862
Epoch 2/10
513/513 [==============================] - 119s 232ms/step - loss: 69175584.0000 - mean_absolute_percentage_error: 45.0501
Epoch 3/10
513/513 [==============================] - 119s 233ms/step - loss: 29720390.0000 - mean_absolute_percentage_error: 27.2025
Epoch 4/10
513/513 [==============================] - 119s 231ms/step - loss: 17268766.0000 - mean_absolute_percentage_error: 21.0369
Epoch 5/10
513/513 [==============================] - 119s 232ms/step - loss: 14466331.0000 - mean_absolute_percentage_error: 19.9057
Epoch 6/10
513/513 [==============================] - 122s 237ms/step -

In [ ]:
#Trying MSE loss instead
history = run_experiment(epochs=10, lambda1=0.8, lambda2=0.2, use_mse=True,
                         metrics=[tf.keras.losses.mean_absolute_percentage_error])

Epoch 1/10
618/618 [==============================] - 10s 9ms/step - loss: 149787.0781 - mean_absolute_percentage_error: 73.6768 - val_loss: 81618.0391 - val_mean_absolute_percentage_error: 50.4059
Epoch 2/10
618/618 [==============================] - 5s 8ms/step - loss: 48777.3008 - mean_absolute_percentage_error: 36.0309 - val_loss: 26372.2559 - val_mean_absolute_percentage_error: 24.9939
Epoch 3/10
618/618 [==============================] - 5s 8ms/step - loss: 17489.2148 - mean_absolute_percentage_error: 20.8270 - val_loss: 12661.5947 - val_mean_absolute_percentage_error: 18.4338
Epoch 4/10
618/618 [==============================] - 5s 7ms/step - loss: 10844.3564 - mean_absolute_percentage_error: 17.6572 - val_loss: 10639.1816 - val_mean_absolute_percentage_error: 18.1014
Epoch 5/10
618/618 [==============================] - 5s 7ms/step - loss: 10035.3301 - mean_absolute_percentage_error: 17.5600 - val_loss: 10522.4854 - val_mean_absolute_percentage_error: 18.3110
Epoch 6/10
618/618

Whole Glocal Net reimplementation

In [ ]:
class